# Bài toán: Dự đoán thời tiết trong ngày:

## Xử lý dữ liệu

In [67]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
import warnings


Đọc dữ liệu đầu vào

In [68]:
data = pd.read_csv('seattle-weather.csv')


In [69]:
data.head()


,date,precipitation,temp_max,temp_min,wind,weather
0,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,2012-01-02,10.9,10.6,2.8,4.5,rain
2,2012-01-03,0.8,11.7,7.2,2.3,rain
3,2012-01-04,20.3,12.2,5.6,4.7,rain
4,2012-01-05,1.3,8.9,2.8,6.1,rain


In [70]:
total = data['weather'].count()
rain = (data['weather']=="rain").sum()
sun = (data['weather']=="sun").sum()
other = total - rain - sun

print(data['weather'].value_counts())
print("Rain: ", rain)
print("Sun: ", sun)
print("Other: ", other)

rain       641
sun        640
fog        101
drizzle     53
snow        26
Name: weather, dtype: int64
Rain:  641
Sun:  640
Other:  180


Tiến hành bỏ cột **Date**

In [71]:
data = data.drop(['date'], axis=1)

Chuyển dữ liệu cột Weather về dạng: Rain: 0, Sun: 1, Other: 2

In [72]:
data['weather'] = np.where(data['weather'] == 'rain',
                           0, np.where(data['weather'] == 'sun', 1, 2))

data.head()


,precipitation,temp_max,temp_min,wind,weather
0,0.0,12.8,5.0,4.7,2
1,10.9,10.6,2.8,4.5,0
2,0.8,11.7,7.2,2.3,0
3,20.3,12.2,5.6,4.7,0
4,1.3,8.9,2.8,6.1,0


Lựa chọn các cột dùng để dự đoán dữ liệu cho cột **Weather**

## Xây dựng, đánh giá mô hình và dự đoán

### Hàm

Hàm tìm bộ tham số tốt nhất cho Logistic Regresstion

In [74]:
def get_hyperparameters(X: np.array, y: np.array):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=9)
    # Định nghĩa  model và tham số
    model = LogisticRegression()
    solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
    penalty = ['none', 'l1', 'l2', 'elasticnet']
    c_values = [100, 10, 1.0, 0.1, 0.01]
    
    grid = dict(solver=solvers, penalty=penalty, C=c_values)
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    grid_search = GridSearchCV(estimator=model, param_grid=grid,
                               n_jobs=-1, cv=cv, scoring='accuracy', error_score=0)
    grid_result = grid_search.fit(X_train, y_train)
    
    return grid_result.best_score_, grid_result.best_params_


Hàm đánh giá độ chính xác mô hình với cross_val_score

In [75]:
# def get_accuracy_by_cross_val_score(X: np.array, y: np.array, best_params={}):
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=1)
#     classifier = LogisticRegression() if not len(best_params) else LogisticRegression(
#         C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'])
#     scores = cross_val_score(
#         classifier, X_train, y_train, cv=5, scoring='accuracy')
#     return scores.mean()

def get_accuracy_by_cross_val_score(X: np.array, y: np.array, best_params={}):
    classifier = LogisticRegression() if not len(best_params) else LogisticRegression(
        C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'])
    scores = cross_val_score(
        classifier, X, y, cv=5, scoring='accuracy')
    return scores.mean()


### Tính độ chính xác của mô hình

Đánh giá độ chính xác của mô hình với tham số mặc định

In [76]:
accuracy_default_params = round(get_accuracy_by_cross_val_score(data.drop(
    ['weather'], axis=1), data['weather']) * 100, 3)

print("Đánh giá độ chính xác của mô hình với tham số mặc định: {}".format(
    accuracy_default_params))


Đánh giá độ chính xác của mô hình với tham số mặc định: 81.933


Tìm bộ tham số tốt nhất cho Logistic Regresstion

In [77]:
warnings.filterwarnings('ignore')

best_score, best_params = get_hyperparameters(data.drop(
    ['weather'], axis=1), data['weather'])

print("Độ chính xác tốt nhất là: {}\nBộ siêu tham số: {}".format(
    best_score, best_params))


Độ chính xác tốt nhất là: 0.829741734881655
Bộ siêu tham số: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}


Đánh giá độ chính xác của mô hình với siêu tham số

In [78]:
accuracy_best_params = round(get_accuracy_by_cross_val_score(data.drop(
    ['weather'], axis=1), data['weather'], best_params) * 100, 3)

print("Đánh giá độ chính xác của mô hình với siêu tham số: {}".format(
    accuracy_best_params))


Đánh giá độ chính xác của mô hình với siêu tham số: 82.687


So sánh độ chính xác giữa 2 phương pháp

In [80]:
print("Độ chính xác chênh lệch giữa mô hình với siêu tham số và mô hình với tham số mặc định: {} %".format(
    round((accuracy_best_params - accuracy_default_params), 3)))


Độ chính xác chênh lệch giữa mô hình với siêu tham số và mô hình với tham số mặc định: 0.754 %


Nhận xét:
- Độ chính xác của mô hình với tham số mặc định: **81.933 %**
- Độ chính xác của mô hình với siêu tham số: **82.687 %**
- Độ chính xác được cải thiện khi mô hình được cấu hình với các siêu tham số **0.754 %**

## Áp dụng các kỹ thuật cải thiện độ chính xác

### Xử lý ngoại lệ

Hàm xử lý ngoại lệ

In [81]:
def outliers(data):
    upper_boundary = data['Age'].mean() + 3 * data['Age'].std()
    lower_boundary = data['Age'].mean() - 3 * data['Age'].std()
    data.loc[data['Age'] >= int(upper_boundary), 'Age'] = int(upper_boundary)

    IQR = data['Fare'].quantile(0.75) - data['Fare'].quantile(0.25)
    upper_bridge = data['Fare'].quantile(0.75) + 3 * IQR
    lower_bridge = data['Fare'].quantile(0.25) - 3 * IQR
    data.loc[data['Fare'] >= int(upper_bridge),'Fare'] = int(upper_bridge)

### Chuẩn hóa

In [82]:
# Chuẩn hóa dữ liệu
# StandardScaler
data_standard_scaler = pd.DataFrame(
    StandardScaler().fit_transform(data), columns=data.columns)
data_standard_scaler['weather'] = data['weather']
# MinMaxScaler
data_minmax = pd.DataFrame(
    MinMaxScaler().fit_transform(data), columns=data.columns)
data_minmax['weather'] = data['weather']
# RobustScaler
data_robust = pd.DataFrame(
    RobustScaler().fit_transform(data), columns=data.columns)
data_robust['weather'] = data['weather']

# Tính độ chính xác sau khi chuẩn hóa dữ liệu
# StandardScaler
accuracy_with_z_score = round(mean_accuracy(data_standard_scaler.drop(
    ['weather'], axis=1), data_standard_scaler['weather'], best_params) * 100, 3)
# MinMaxScaler
accuracy_with_min_max = round(mean_accuracy(data_minmax.drop(
    ['weather'], axis=1), data_minmax['weather'], best_params) * 100, 3)
# RobustScaler
accuracy_with_robust = round(mean_accuracy(data_robust.drop(
    ['weather'], axis=1), data_robust['weather'], best_params) * 100, 3)

# Kết quả
print("Độ chính xác mô hình với siêu tham số và chuẩn hóa: {} %".format(accuracy_with_z_score))
print("Độ chính xác mô hình với siêu tham số và chuẩn hóa: {} %".format(accuracy_with_min_max))
print("Độ chính xác mô hình với siêu tham số và chuẩn hóa: {} %".format(accuracy_with_robust))
print("Độ chênh lệch chính xác khi không có chuẩn hóa và có chuẩn hóa là:",
      round((accuracy_with_z_score - accuracy_best_params), 3), "%")
print("Độ chênh lệch chính xác khi không có chuẩn hóa và có chuẩn hóa là:",
      round((accuracy_with_min_max - accuracy_best_params), 3), "%")
print("Độ chênh lệch chính xác khi không có chuẩn hóa và có chuẩn hóa là:",
      round((accuracy_with_robust - accuracy_best_params), 3), "%")


NameError: name 'mean_accuracy' is not defined

### Giảm chiều dữ liệu

In [ ]:
# data_pca = pd.DataFrame(
#     PCA(n_components=3).fit_transform(data), columns=data.columns)
# data_pca['weather'] = data['weather']

# accuracy_with_pca = round(mean_accuracy(data_pca.drop(
#     ['weather'], axis=1), data_pca['weather'], best_params) * 100, 3)

# print(accuracy_with_pca - accuracy_best_params)


# def test(X, y, best_params):
#     list_rs = []
#     for i in range(10):
#         X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=i)
#         pca = PCA(n_components=3)
#         X_train = pca.fit_transform(X_train)
#         X_test = pca.transform(X_test)
#         classifier = LogisticRegression(C = best_params['C'], solver = best_params['solver'], penalty = best_params['penalty'])
#         classifier.fit(X_train, y_train)
#         y_pred = classifier.predict(X_test)
#         accuracy = accuracy_score(y_test,y_pred)
#         list_rs.append(accuracy)
#     return np.array(list_rs).mean()

# rs_with_pca = test(data.drop(['weather'], axis=1), data['weather'], best_params)
# print("Độ chính xác mô hình với siêu tham số và pca:", round(rs_with_pca* 100,4),"%")

# def mean_accuracy(X: np.array, y: np.array, best_params={}):
#     accuracy_score_array = []

#     for i in range(10):
#         X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size=0.3, random_state=i)
#         classifier = LogisticRegression() if not len(best_params) else LogisticRegression(
#             C=best_params['C'], penalty=best_params['penalty'], solver=best_params['solver'])
#         classifier.fit(X_train, y_train)
#         y_pred = classifier.predict(X_test)
#         accuracy_score_array.append(accuracy_score(y_test, y_pred))

#     return np.array(accuracy_score_array).mean()

